#### Getting data from the World Bank to build a dataframe

In [3]:
# imports
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot
import seaborn as sns

from functools import reduce

-----
## Read in and Aggregate Agriculture/Rural Development Data

In [4]:
# read in and format a csv file from World Bank Indicators
def world_bank_csv(name, filepath, indicator_name):
    '''
    Reads in csv file from World Bank
    Renames columns
    Selects 2017 column
    prepares to be merged with other dataframes
    '''
    name = pd.read_csv('../datasets/' + filepath, skiprows = 4)
    name.drop(columns = ['Unnamed: 66'], inplace = True)
    
    #selecting useful columns
    name = name[['Country Name','Country Code','2017']].copy()
    
    # rename to specify 
    name.rename(columns = {'2017': indicator_name}, inplace = True)
    
    return name

In [5]:
rural_pop_17 = world_bank_csv('rural_pop_17', 'API_SP.RUR.TOTL.ZS_DS2_en_csv_v2_4261416.csv','rural_pop_percent_2017')

In [6]:
rural_pop_17.head()

,Country Name,Country Code,rural_pop_percent_2017
0,Aruba,ABW,56.707000
1,Africa Eastern and Southern,AFE,64.667627
2,Afghanistan,AFG,74.750000
3,Africa Western and Central,AFW,53.980666
4,Angola,AGO,35.161000


In [7]:
names = ['rural_pop_percent_17', 'food_production_index_17', 'ag_land_area_17', 'arable_land_percent_2017',
         'net_migration_2017','hiv_prevalence_2017','mat_mortality_ratio_2017','under5_mortality_ratio_2017',
        'tubercul_incidence_2017','elec_access_2017','ren_energy_percent_2017','ffuel_energy_percent_2017',
        'co2_emissions_2017','pop_air_pollution_2017','foreign_dir_inv_2017','atm_access_2017','legal_rights_index_2017',
        'adol_fertility_rate_2017','fem_labor_part_rate_2017','male_labor_part_rate_2017','fertility_rate_2017',
        'dpt_immuniz_rate_2017','undernourished_rate_2017','cell_subscriptions_per100','internet_per_mil_2017',
        'military_exp_2017','women_seats_percent_2017','male_bus_start_2017','female_bus_start_2017','patent_apps_2017',
        'sci_articles_2017','pop_density_2017']

file_paths = ['API_SP.RUR.TOTL.ZS_DS2_en_csv_v2_4261416.csv', 'API_AG.PRD.FOOD.XD_DS2_en_csv_v2_4254742.csv', 'API_AG.LND.AGRI.ZS_DS2_en_csv_v2_4254639.csv',
            'API_AG.LND.ARBL.ZS_DS2_en_csv_v2_4252676.csv','API_SM.POP.NETM_DS2_en_csv_v2_4250793.csv','API_SH.DYN.AIDS.ZS_DS2_en_csv_v2_4250791.csv',
            'API_SH.STA.MMRT_DS2_en_csv_v2_4252399.csv','API_SH.DYN.MORT_DS2_en_csv_v2_4252415.csv',
            'API_SH.TBS.INCD_DS2_en_csv_v2_4250622.csv','API_SP.RUR.TOTL.ZS_DS2_en_csv_v2_4261416.csv','API_EG.FEC.RNEW.ZS_DS2_en_csv_v2_4251598.csv',
            'API_EG.USE.COMM.FO.ZS_DS2_en_csv_v2_4250919.csv','API_EN.ATM.CO2E.PC_DS2_en_csv_v2_4251354.csv','API_EN.ATM.PM25.MC.M3_DS2_en_csv_v2_4251710.csv',
            'API_BX.KLT.DINV.CD.WD_DS2_en_csv_v2_4250821.csv','API_FB.ATM.TOTL.P5_DS2_en_csv_v2_4260830.csv','API_IC.LGL.CRED.XQ_DS2_en_csv_v2_4261026.csv',
            'API_SP.ADO.TFRT_DS2_en_csv_v2_4252410.csv','API_SL.TLF.CACT.FE.ZS_DS2_en_csv_v2_4250833.csv','API_SL.TLF.CACT.MA.ZS_DS2_en_csv_v2_4251123.csv',
            'API_SP.DYN.TFRT.IN_DS2_en_csv_v2_4252390.csv','API_SH.IMM.IDPT_DS2_en_csv_v2_4261936.csv','API_SN.ITK.DEFC.ZS_DS2_en_csv_v2_4251163.csv',
            'API_IT.CEL.SETS.P2_DS2_en_csv_v2_4251970.csv','API_IT.NET.SECR.P6_DS2_en_csv_v2_4251938.csv','API_MS.MIL.XPND.GD.ZS_DS2_en_csv_v2_4349031.csv',
            'API_SG.GEN.PARL.ZS_DS2_en_csv_v2_4353272.csv','API_IC.REG.DURS.MA_DS2_en_csv_v2_4353601.csv','API_IC.REG.DURS.FE_DS2_en_csv_v2_4353597.csv',
            'API_IP.PAT.RESD_DS2_en_csv_v2_4353337.csv','API_IP.JRN.ARTC.SC_DS2_en_csv_v2_4353653.csv','API_EN.POP.DNST_DS2_en_csv_v2_4353230.csv']           

In [8]:
def merged_table(names_list, path_list):
    '''
    This function iterates through the various World Bank Indicators
    pulls in csv data and filters for the 2017 column
    and merges the sub-data frames into one merged copy
    '''
    data_frames = []
    
    # Creating data frames for each indicator
    # appends the name of each new data frame to a list to be used in the merge
    
    for i in range(len(names_list)):
        df_name = world_bank_csv(names[i], file_paths[i], names[i]) # calling the function that was created previously
        data_frames.append(df_name)
        time.sleep(1)
        
    # the reduce function code was adapted from everestial007 response on stack overflow (https://stackoverflow.com/questions/44327999/python-pandas-merge-multiple-dataframes)
    # merging the dataframes
    merged_df = reduce(lambda left,right: pd.merge(left,right,
                                                   left_index = True,
                                                   right_index = True,
                                                   suffixes=(None, '_right'),
                                                   how='inner',
                                                  ),
                                                   data_frames)
    
    # remove any duplicate columns
    merged_df = merged_df[[col for col in merged_df if "right" not in col]]
    
    return merged_df

In [9]:
merged_df = merged_table(names, file_paths)

In [10]:
merged_df

,Country Name,Country Code,rural_pop_percent_17,food_production_index_17,ag_land_area_17,arable_land_percent_2017,net_migration_2017,hiv_prevalence_2017,mat_mortality_ratio_2017,under5_mortality_ratio_2017,...,undernourished_rate_2017,cell_subscriptions_per100,internet_per_mil_2017,military_exp_2017,women_seats_percent_2017,male_bus_start_2017,female_bus_start_2017,patent_apps_2017,sci_articles_2017,pop_density_2017
0,Aruba,ABW,56.707000,NaN,11.111111,11.111111,1004.0,NaN,NaN,NaN,...,NaN,NaN,977.591329,NaN,NaN,NaN,NaN,NaN,NaN,585.338889
1,Africa Eastern and Southern,AFE,64.667627,NaN,44.042629,8.231181,-791282.0,5.371653,398.0,64.243994,...,23.294809,64.881587,894.612659,1.503970,28.705923,29.250000,29.326923,NaN,19259.19,42.303709
2,Afghanistan,AFG,74.750000,99.169998,58.067580,11.792727,-314602.0,0.100000,638.0,64.800000,...,23.000000,65.929134,43.448181,0.945227,27.710843,8.000000,9.000000,NaN,91.89,55.595550
3,Africa Western and Central,AFW,53.980666,NaN,39.930699,11.224018,-1022839.0,1.563411,717.0,102.032086,...,12.005043,84.672561,109.743511,0.883374,15.279107,17.814545,17.996364,NaN,7907.50,46.847251
4,Angola,AGO,35.161000,103.419998,45.237371,3.930376,32066.0,1.900000,241.0,80.600000,...,15.400000,44.686107,10.799292,2.507985,30.454545,36.000000,36.000000,NaN,55.50,23.916555
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261,Kosovo,XKX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.795720,NaN,5.500000,5.500000,NaN,261.45,164.508405
262,"Yemen, Rep.",YEM,63.984000,91.220001,44.297403,2.079095,-150000.0,0.100000,164.0,60.200000,...,46.600000,55.175024,3.377066,NaN,0.000000,40.000000,41.000000,15.0,126.37,52.720441
263,South Africa,ZAF,34.150000,105.449997,79.417850,9.892094,727026.0,19.400000,119.0,34.600000,...,5.500000,155.232396,9429.158882,1.030061,41.836735,45.000000,45.000000,728.0,12846.08,46.995483
264,Zambia,ZMB,57.024000,124.970001,32.063923,5.111718,-40000.0,11.500000,213.0,64.600000,...,NaN,79.736910,42.364816,1.309596,17.964072,8.500000,8.500000,12.0,209.45,22.671287


In [11]:
merged_df.to_csv('../datasets/merged_df.csv', index=False) # saved combined data frame to csv
# ready for further cleaning

### Add target Health Development Index data

In [12]:
#  ATTENTION:  for now start here (after running imports) to work on merging dataframes
# Peter already did all the work above and successfully saved a csv in datasets

target = pd.read_csv('../datasets/hdi_human_development_index.csv')

In [13]:
target = target[['country', '2017']].copy()

In [14]:
target.rename(columns={'country': 'Country Name', '2017': 'HDI'}, inplace=True)

In [15]:
target['Country Name'] = target['Country Name'].replace(['Bahamas','Brunei', 'Gambia', 'Hong Kong, China', 'Iran', 'South Korea', 'Lao', 'Palestine', 'Russia', 'Syria', 'Turkey', 'Venezuela', 'Yemen'],
                                                        ['Bahamas, The', 'Brunei Darussalam', 'Gambia, The', 'Hong Kong SAR, China', 'Iran, Islamic Rep.', 'Korea, Rep.', 'Lao PDR', 'West Bank and Gaza', 'Russian Federation',
                                                        'Syrian Arab Republic', 'Turkiye', 'Venezuela, RB', 'Yemen, Rep.'])

In [16]:
df = pd.read_csv('../datasets/merged_df.csv')

In [17]:
# doing this way (default is 'inner') lowers us to 174 rows.  Some countries have slightly different names in the HDI dataset,
# and the WB data has some non-countries in it, i.e. 'High Income' and 'Heavily Indebted Poor Countries'

merged = pd.merge(df, target)

In [18]:
merged

,Country Name,Country Code,rural_pop_percent_17,food_production_index_17,ag_land_area_17,arable_land_percent_2017,net_migration_2017,hiv_prevalence_2017,mat_mortality_ratio_2017,under5_mortality_ratio_2017,...,cell_subscriptions_per100,internet_per_mil_2017,military_exp_2017,women_seats_percent_2017,male_bus_start_2017,female_bus_start_2017,patent_apps_2017,sci_articles_2017,pop_density_2017,HDI
0,Afghanistan,AFG,74.750,99.169998,58.067580,11.792727,-314602.0,0.1,638.0,64.8,...,65.929134,43.448181,0.945227,27.710843,8.0,9.0,NaN,91.89,55.595550,0.509
1,Angola,AGO,35.161,103.419998,45.237371,3.930376,32066.0,1.9,241.0,80.6,...,44.686107,10.799292,2.507985,30.454545,36.0,36.0,NaN,55.50,23.916555,0.582
2,Albania,ALB,40.617,103.980003,42.670839,22.149635,-69998.0,0.1,15.0,9.4,...,125.710352,443.020376,1.108696,27.857143,5.0,5.0,16.0,149.54,104.870693,0.792
3,Andorra,AND,11.850,NaN,40.042553,1.744681,NaN,NaN,NaN,3.0,...,104.332411,3415.717495,NaN,32.142857,NaN,NaN,NaN,2.01,163.823404,0.867
4,United Arab Emirates,ARE,13.752,104.180000,5.404112,0.626584,200000.0,0.1,3.0,7.2,...,208.978600,1285.099111,NaN,22.500000,8.0,9.0,63.0,2899.67,133.584990,0.889
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182,Samoa,WSM,81.548,102.620003,26.749117,11.484099,-14013.0,NaN,43.0,18.1,...,63.583173,296.890836,NaN,10.000000,9.0,9.0,NaN,10.22,69.031095,0.709
183,"Yemen, Rep.",YEM,63.984,91.220001,44.297403,2.079095,-150000.0,0.1,164.0,60.2,...,55.175024,3.377066,NaN,0.000000,40.0,41.0,15.0,126.37,52.720441,0.468
184,South Africa,ZAF,34.150,105.449997,79.417850,9.892094,727026.0,19.4,119.0,34.6,...,155.232396,9429.158882,1.030061,41.836735,45.0,45.0,728.0,12846.08,46.995483,0.707
185,Zambia,ZMB,57.024,124.970001,32.063923,5.111718,-40000.0,11.5,213.0,64.6,...,79.736910,42.364816,1.309596,17.964072,8.5,8.5,12.0,209.45,22.671287,0.582


In [19]:
merged.to_csv('../datasets/merged_df_mon.csv', index=False)